In [3]:
# from CBAStats.Player import *
# from CBAStats.Team import *
# from CBAStats.Player import stats_output
import datetime
import Scraper
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
import pymysql

In [4]:
print('最近一次运行时间：',datetime.datetime.now())

最近一次运行时间： 2020-07-30 01:13:28.892604


# 更新数据

In [5]:
# 定义文件路径，如果文件已存在则会覆盖旧文件
schedule_output = f'../StatsData/GamesSchedulePage_{datetime.date.today()}.csv'
game_stats_output = f'../StatsData/All_Games_Stats_{datetime.date.today()}.csv'
teams_stats_output = f'../StatsData/TeamsAdvancedStats_{datetime.date.today()}.csv'
players_stats_output = f'../StatsData/PlayersAdvancedStats_{datetime.date.today()}.csv'

for output in [schedule_output,game_stats_output,teams_stats_output,players_stats_output]:
    path = Path(output)
    print(f'Checking {output}.')
    # check if folder is reachable
    assert (path.parent.exists()),f"Folder '{path.parent}' does not exist."
    # check if file already exists
    if path.exists():
        print(f'\tFile {path} already exists! It will be overwritten.')
    print('\tPass!')

Checking ../StatsData/GamesSchedulePage_2020-07-30.csv.
	Pass!
Checking ../StatsData/All_Games_Stats_2020-07-30.csv.
	Pass!
Checking ../StatsData/TeamsAdvancedStats_2020-07-30.csv.
	Pass!
Checking ../StatsData/PlayersAdvancedStats_2020-07-30.csv.
	Pass!


In [6]:
# only supports csv file for now
Scraper.scrape_sina_schedule(output_path=schedule_output)
Scraper.scrape_game_details(input_file=schedule_output,
                            output_file=game_stats_output)

开始爬取所有赛程、数据统计链接.
所有赛程、数据统计链接将会存在 ../StatsData/GamesSchedulePage_2020-07-30.csv.
所有赛程、数据统计链接已保存在 ../StatsData/GamesSchedulePage_2020-07-30.csv.
从../StatsData/GamesSchedulePage_2020-07-30.csv读取所有赛程、数据统计链接
读取完毕
抓取比赛  2019202000001   广东   辽宁   2020-07-30 01:13:33.393067
抓取比赛  2019202000002   江苏   浙江   2020-07-30 01:13:37.455291
抓取比赛  2019202000003   四川   新疆   2020-07-30 01:13:40.258791
抓取比赛  2019202000004   山东   八一   2020-07-30 01:13:41.672635
抓取比赛  2019202000005   北控   深圳   2020-07-30 01:13:44.692620
抓取比赛  2019202000006   青岛   吉林   2020-07-30 01:13:46.611714
抓取比赛  2019202000007   广厦   福建   2020-07-30 01:13:47.630847
抓取比赛  2019202000008   上海   同曦   2020-07-30 01:13:49.730520
抓取比赛  2019202000009   天津   北京   2020-07-30 01:13:52.874164
抓取比赛  2019202000010   广州   山西   2020-07-30 01:13:56.231327
抓取比赛  2019202000011   辽宁   广厦   2020-07-30 01:13:58.386868
抓取比赛  2019202000012   北控   上海   2020-07-30 01:14:00.442560
抓取比赛  2019202000013   同曦   江苏   2020-07-30 01:14:03.725661
抓取比赛  2019202000014   山西 

抓取比赛  2019202000137   广州   辽宁   2020-07-30 01:19:40.138015
抓取比赛  2019202000138   北京   深圳   2020-07-30 01:19:44.163995
抓取比赛  2019202000139   八一   江苏   2020-07-30 01:19:45.751752
抓取比赛  2019202000140   吉林   新疆   2020-07-30 01:19:49.259177
抓取比赛  2019202000141   福建   北控   2020-07-30 01:19:52.747069
抓取比赛  2019202000142   吉林   上海   2020-07-30 01:19:53.669931
抓取比赛  2019202000143   北京   广州   2020-07-30 01:19:55.189246
抓取比赛  2019202000144   浙江   八一   2020-07-30 01:19:58.761970
抓取比赛  2019202000145   辽宁   同曦   2020-07-30 01:20:00.829451
抓取比赛  2019202000146   广东   广厦   2020-07-30 01:20:02.728168
抓取比赛  2019202000147   山东   江苏   2020-07-30 01:20:05.409596
抓取比赛  2019202000148   山西   新疆   2020-07-30 01:20:07.150552
抓取比赛  2019202000149   青岛   深圳   2020-07-30 01:20:10.890180
抓取比赛  2019202000150   四川   天津   2020-07-30 01:20:11.771764
抓取比赛  2019202000151   吉林   同曦   2020-07-30 01:20:13.526920
抓取比赛  2019202000152   北控   广州   2020-07-30 01:20:17.309147
抓取比赛  2019202000153   广厦   八一   2020-07-30 01:20:19.4317

抓取比赛  2019202000276   福建   八一   2020-07-30 01:25:52.848781
抓取比赛  2019202000277   吉林   新疆   2020-07-30 01:25:55.467338
抓取比赛  2019202000278   广州   青岛   2020-07-30 01:25:57.866941
抓取比赛  2019202000279   北京   深圳   2020-07-30 01:26:01.778500
抓取比赛  2019202000280   浙江   北控   2020-07-30 01:26:04.672864
抓取比赛  2019202000281   广东   山东   2020-07-30 01:26:05.680961
抓取比赛  2019202000282   同曦   山西   2020-07-30 01:26:08.098840
抓取比赛  2019202000283   吉林   八一   2020-07-30 01:26:12.279902
抓取比赛  2019202000284   辽宁   新疆   2020-07-30 01:26:15.844072
抓取比赛  2019202000285   广州   四川   2020-07-30 01:26:17.129304
抓取比赛  2019202000286   上海   深圳   2020-07-30 01:26:20.703449
抓取比赛  2019202000287   北京   江苏   2020-07-30 01:26:23.570390
抓取比赛  2019202000288   浙江   天津   2020-07-30 01:26:27.217109
抓取比赛  2019202000289   广厦   北控   2020-07-30 01:26:29.463993
抓取比赛  2019202000290   福建   青岛   2020-07-30 01:26:32.474655
抓取比赛  2019202000291   吉林   山西   2020-07-30 01:26:33.899454
抓取比赛  2019202000292   江苏   上海   2020-07-30 01:26:35.7831

抓取比赛  2019202000415   广厦   广东   2020-07-30 01:31:57.976047
抓取比赛  2019202000416   山东   新疆   2020-07-30 01:32:00.258675
抓取比赛  2019202000417   八一   上海   2020-07-30 01:32:03.582137
抓取比赛  2019202000418   青岛   北控   2020-07-30 01:32:05.042850
抓取比赛  2019202000419   同曦   吉林   2020-07-30 01:32:08.931281
抓取比赛  2019202000420   辽宁   福建   2020-07-30 01:32:11.011577
抓取比赛  2019202000421   江苏   四川   2020-07-30 01:32:14.037126
抓取比赛  2019202000422   广州   北京   2020-07-30 01:32:16.757471
抓取比赛  2019202000423   浙江   山西   2020-07-30 01:32:17.983636
抓取比赛  2019202000424   上海   吉林   2020-07-30 01:32:21.012588
抓取比赛  2019202000425   天津   八一   2020-07-30 01:32:22.509841
抓取比赛  2019202000426   北控   辽宁   2020-07-30 01:32:25.564737
抓取比赛  2019202000427   广厦   深圳   2020-07-30 01:32:28.593983
抓取比赛  2019202000428   北京   四川   2020-07-30 01:32:32.931739
抓取比赛  2019202000429   山西   青岛   2020-07-30 01:32:35.003062
抓取比赛  2019202000430   浙江   广东   2020-07-30 01:32:38.210666
抓取比赛  2019202000431   福建   新疆   2020-07-30 01:32:41.3301

# 载入爬取的数据

In [7]:
df = pd.read_csv(game_stats_output,dtype={'号码':'str','Game_ID':'str'})
df

,球员,号码,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,抢断,失误,...,得分,Game_ID,球队,对手,2分中,2分投,3分中,3分投,罚球中,罚球投
0,布鲁克斯,6,36,1,0,3,9,3,2,1,...,27,2019202000001,广东,辽宁,8,10,2,10,5,5
1,任骏飞,20,30,1,1,1,1,3,3,0,...,19,2019202000001,广东,辽宁,5,8,3,6,0,0
2,易建联,9,35,1,1,11,2,2,1,1,...,16,2019202000001,广东,辽宁,7,8,0,0,2,4
3,赵睿,10,31,1,2,5,4,3,2,0,...,10,2019202000001,广东,辽宁,5,9,0,3,0,1
4,胡明轩,3,13,1,0,1,0,1,0,1,...,3,2019202000001,广东,辽宁,0,1,1,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15860,白杰,16,0,0,0,0,0,0,0,0,...,0,2019202000461,浙江,深圳,0,0,0,0,0,0
15861,陆文博,8,0,0,0,0,0,0,0,0,...,0,2019202000461,浙江,深圳,0,0,0,0,0,0
15862,程帅澎,6,0,0,0,0,0,0,0,0,...,0,2019202000461,浙江,深圳,0,0,0,0,0,0
15863,张大宇,24,0,0,0,0,0,0,0,0,...,0,2019202000461,浙江,深圳,0,0,0,0,0,0


# Insert Records

In [8]:
from sqlalchemy import create_engine
import pymysql

user_name = 'master'
passcode = 'Pw#cbashuju0131'
endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
database = 'CBA_Data'
engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')
connection= engine.connect()

df.to_sql(name='PlayerStatsPerGame',con=connection,index=False,if_exists='replace')

connection.close()